<a href="https://www.kaggle.com/code/mosaabseta/bone-fractures?scriptVersionId=143990968" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
datagen = ImageDataGenerator(
    rescale=1.0/255.0, 
    rotation_range=20,  
    width_shift_range=0.2, 
    height_shift_range=0.2,  
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True,  
    fill_mode='nearest'
)

In [ ]:
batch_size = 20
train_data_dir = '/kaggle/input/bone-fracture-detection-using-xrays/archive (6)/train'
validation_data_dir = '/kaggle/input/bone-fracture-detection-using-xrays/archive (6)/val'


train_generator = datagen.flow_from_directory( train_data_dir,
    target_size=(224, 224), 
    batch_size=batch_size,
    class_mode='binary',  
    shuffle=True )
validation_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False  
)

**simple CNN**

ResNet50

In [ ]:
resModel = Sequential()
resModel.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
resModel.add(Dense(1, activation='sigmoid'))
# Unfreeze some layers for fine-tuning
for layer in resModel.layers[0].layers[-50:]:
    layer.trainable = True

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau


optimizer = Adam(learning_rate=0.001)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

resModel.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 20
history = resModel.fit(train_generator, epochs=epochs, validation_data=validation_generator, callbacks=[reduce_lr])

# Save the trained model
resModel.save('/kaggle/working/bone_fractures_trained_model_resNet.h5')
